<a href="https://colab.research.google.com/github/Aatika1/openai-API/blob/main/chatbot0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai

In [2]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"

gemini_api_key = userdata.get('GEMINI_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

llm.invoke("greet me")

AIMessage(content='Hello there! How can I help you today? \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3a17ff43-0d49-4323-b571-31591c0c1cb9-0', usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}})

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [6]:
# # # Example usage of add_messages
new_messages_1 = [{"role": "user", "content": "Hello"}]
new_messages_2 = [{"role": "ai", "content": "Hi there!"}]

messages = add_messages(new_messages_1, new_messages_2)


print("Updated State 1:", messages)

# from typing import Annotated
# from typing_extensions import TypedDict
# from langgraph.graph import StateGraph

# class State(TypedDict):
#     messages: Annotated[list, add_messages]

# builder = StateGraph(State)
# builder.add_node("chatbot", lambda state: {"messages": [("assistant", "Hello")]})
# builder.set_entry_point("chatbot")
# builder.set_finish_point("chatbot")
# graph = builder.compile()
# graph.invoke({"messages": [{"role": "user", "content": "Hello"}]})

Updated State 1: [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='390565a0-21a6-409e-8c86-8de04068cde9'), AIMessage(content='Hi there!', additional_kwargs={}, response_metadata={}, id='546d49ca-6837-4844-8b12-338f1c7c4698')]


In [5]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [6]:
graph_builder.add_edge(START,"chatbot")

In [7]:
graph_builder.add_edge("chatbot",END)

In [8]:
graph=graph_builder.compile()

In [9]:
for bussines in graph.stream({"messages": ["user","Hello I am abuzar"]}):
  print("bussines:", list(bussines.values())[0]["messages"][-1].content)

bussines: Hello Abuzar! Nice to meet you.  What can I do for you today? 



In [10]:
def graph_updates(user_input:str):
  for bussines in graph.stream({"messages": [("user",user_input)]}):
    for value in bussines.values():
        print("Assistant: ", value["messages"][-1].content)

while True:
  try:
    user_input=input("user: ")
    if user_input.lower() in ["exit","quit","bye"]:
      print("Nice to meet you")
      break

    graph_updates(user_input)
  except:
    #fallback if input() is not available
    user_input="what do you know about langGraph"
    print("user" + user_input)
    graph_updates(user_input)
    break




user: hi
Assistant:  Hi there! How can I help you today? 

user: exit
Nice to meet you


In [11]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [12]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=2)



In [14]:
search_tool.invoke("current cm of sindh pakistan")

[{'url': 'https://en.wikipedia.org/wiki/Chief_Minister_of_Sindh',
  'content': 'cm.sindh.gov.pk. The chief minister of Sindh (Sindhi: وزيراعلي, Urdu: وزیر اعلىٰ — Wazīr-e Aʿlá), is the elected head of government of Sindh and serves alongside the Chief Secretary. Syed Murad Ali Shah is the current Chief Minister of Sindh, serving as a Government of Sindh since 26 February 2024. The chief minister is'},
 {'url': 'https://www.nation.com.pk/27-Feb-2024/profile-of-newly-elected-sindh-cm-murad-ali-shah',
  'content': "KARACHI - Newly elected Sindh Chief Min\xadister (CM) Sayed Murad Ali Shah belongs to a political family of Sindh's Jamshoro district. He was born on November 8, 1962 at Karachi. His father Syed Abdullah Shah was also served as chief minister of Sindh province from October 21, 1993 to November 6, 1996, during the Pakistan People's Party's"}]

In [15]:
tools=[search_tool]

In [16]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

In [17]:
llm


ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c61cf0563b0>, default_metadata=())

In [18]:
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c61cf0563b0>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [19]:
llm_with_tools.invoke("say welcome to zahreen")

AIMessage(content='Welcome, Zahreen! 👋  😊 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-cf75767f-3d5c-4dca-97f5-a008d71fbb21-0', usage_metadata={'input_tokens': 79, 'output_tokens': 8, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}})

In [20]:
tools_by_name = {tool.name: tool for tool in tools}
print(type(tools_by_name))
print(tools_by_name)



<class 'dict'>
{'tavily_search_results_json': TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))}


In [21]:
tools_by_name["tavily_search_results_json"]


TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))

In [22]:
from typing import Annotated


from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]

llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile()

In [23]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "bye"]:
            print("Nice to meet yoy!")
            break

        graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        graph_updates(user_input)
        break

User: what is current weather of ratodero
Assistant:  
Assistant:  [{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Ratodero', 'region': 'Sindh', 'country': 'Pakistan', 'lat': 27.8, 'lon': 68.3, 'tz_id': 'Asia/Karachi', 'localtime_epoch': 1731425386, 'localtime': '2024-11-12 20:29'}, 'current': {'last_updated_epoch': 1731424500, 'last_updated': '2024-11-12 20:15', 'temp_c': 28.1, 'temp_f': 82.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 318, 'wind_dir': 'NW', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 24, 'cloud': 2, 'feelslike_c': 26.5, 'feelslike_f': 79.6, 'windchill_c': 28.1, 'windchill_f': 82.6, 'heatindex_c': 26.5, 'heatindex_f': 79.6, 'dewpoint_c': 5.5, 'dewpoint_f': 41.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 10.3, 'gust_kph': 16.6}}"}, {"url": "https://www.weat